In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import sklearn
from sklearn.model_selection import train_test_split

import tensorflow as tf
tf.__version__

'2.3.1'

In [8]:
np.random.seed(10)
tf.compat.v1.set_random_seed(10)

In [9]:
# Cargo los csv
df_global = pd.read_csv("./unified-lightcurves-global.csv").iloc[:, 1:]

display(df_global)

,kepid,kepler_name,kepoi_name,koi_disposition,light_flux_0001,light_flux_0002,light_flux_0003,light_flux_0004,light_flux_0005,light_flux_0006,...,light_flux_0992,light_flux_0993,light_flux_0994,light_flux_0995,light_flux_0996,light_flux_0997,light_flux_0998,light_flux_0999,light_flux_1000,light_flux_1001
0,10797460.0,Kepler-227 b,K00752.01,1,0.832189,0.435614,1.357075,1.108348,1.650665,1.129287,...,0.713763,1.274681,1.154911,2.088258,1.059249,0.848340,1.142176,0.961584,0.966640,1.044353
1,10811496.0,NaN,K00753.01,0,0.966682,0.964235,0.970539,0.937743,0.975104,1.039065,...,0.943850,0.965717,0.990491,0.982717,0.985906,0.961670,1.043547,0.983126,1.040699,1.040926
2,10848459.0,NaN,K00754.01,0,1.034104,0.982172,1.048199,1.078219,0.983764,0.992431,...,0.946015,1.010975,1.011789,1.055202,1.038121,1.044473,1.071489,1.053268,1.033189,1.005258
3,10854555.0,Kepler-664 b,K00755.01,1,0.975071,1.225692,1.269809,1.771289,1.312162,1.097620,...,1.335065,1.179038,1.002768,1.017255,0.823422,0.561212,1.035374,0.835232,1.615619,1.464552
4,10872983.0,Kepler-228 d,K00756.01,1,1.394998,1.185855,0.932441,1.230771,0.940595,0.909058,...,0.864800,0.850251,1.086740,1.073562,0.930443,0.857681,1.232381,1.297968,1.208948,1.079670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6090,10031643.0,NaN,K07984.01,0,0.175475,1.504283,1.318089,0.769620,0.666730,1.008246,...,1.378201,1.927068,1.984925,1.130322,1.202864,0.781321,0.045358,0.647927,1.207362,1.101768
6091,10090151.0,NaN,K07985.01,0,0.747230,0.915918,1.031489,0.989710,1.021699,0.867820,...,0.755320,0.876177,0.913214,0.960485,1.017231,0.789195,0.809078,0.774575,0.872496,0.802597
6092,10128825.0,NaN,K07986.01,0,0.714833,1.610737,0.196081,0.533665,1.053996,0.873009,...,1.082519,1.101279,1.060959,1.446257,1.603752,0.875442,1.295145,1.852598,0.230911,0.803615
6093,10147276.0,NaN,K07987.01,0,1.486451,1.733356,0.993478,0.776943,0.226574,0.407813,...,0.898611,1.483079,1.160034,0.921238,1.509204,-0.560163,0.666612,1.113881,2.175872,2.393636


In [10]:
global_fluxes = df_global.iloc[:,4:]
global_labels = df_global.iloc[:,3]

## Particionado de datos

In [11]:
x_global_train, x_global_test, y_global_train, y_global_test = train_test_split(
    global_fluxes, global_labels, test_size=0.3, random_state=11)

In [12]:
print(x_global_train.shape, x_global_test.shape, y_global_train.shape, y_global_test.shape)

(4266, 1001) (1829, 1001) (4266,) (1829,)


In [13]:
#Definimos las dimensiones
n_outputs = 1
n_global_timesteps, n_global_features  = x_global_train.shape[0], x_global_train.shape[1]

In [14]:
print(n_global_timesteps, n_global_features)

4266 1001


In [15]:
#Expandimos las dimensiones de train 
x_global_extended_train = np.expand_dims(x_global_train,axis=-1)
y_global_extended_train = np.array(y_global_train)

In [16]:
print(x_global_train.shape, y_global_train.shape)
print(x_global_extended_train.shape, y_global_extended_train.shape)

(4266, 1001) (4266,)
(4266, 1001, 1) (4266,)


In [17]:
#Expandimos las dimensiones de test
x_global_extended_test = np.expand_dims(x_global_test,axis=-1)
y_global_extended_test = np.array(y_global_test)

In [18]:
print(x_global_test.shape, y_global_test.shape)
print(x_global_extended_test.shape, y_global_extended_test.shape)

(1829, 1001) (1829,)
(1829, 1001, 1) (1829,)


## Construcción Red Neuronal

In [19]:
inputGlobalView = tf.keras.layers.Input(shape=(n_global_features, 1))
inputGlobalView.set_shape([n_global_timesteps, n_global_features, 1])  # 4266 x 1001


CG1 = tf.keras.layers.Conv1D(filters=16, kernel_size=5, activation='relu')(inputGlobalView)
CG2 = tf.keras.layers.Conv1D(filters=16, kernel_size=5, activation='relu')(CG1)

MG1 = tf.keras.layers.MaxPooling1D(pool_size=3, strides=2)(CG2)

CG3 = tf.keras.layers.Conv1D(filters=32, kernel_size=5, activation='relu')(MG1)
CG4 = tf.keras.layers.Conv1D(filters=32, kernel_size=5, activation='relu')(CG3)

MG2 = tf.keras.layers.MaxPooling1D(pool_size=2, strides=2)(CG4)

CG5 = tf.keras.layers.Conv1D(filters=64, kernel_size=5, activation='relu')(MG2)
CG6 = tf.keras.layers.Conv1D(filters=64, kernel_size=5, activation='relu')(CG5)

MG3 = tf.keras.layers.MaxPooling1D(pool_size=2, strides=2)(CG6)

CG7 = tf.keras.layers.Conv1D(filters=128, kernel_size=5, activation='relu')(MG3)
CG8 = tf.keras.layers.Conv1D(filters=128, kernel_size=5, activation='relu')(CG7)

MG4 = tf.keras.layers.MaxPooling1D(pool_size=2, strides=2)(CG8)

CG9 = tf.keras.layers.Conv1D(filters=256, kernel_size=5, activation='relu')(MG4)
CG10 = tf.keras.layers.Conv1D(filters=256, kernel_size=5, activation='relu')(CG9)

MG5 = tf.keras.layers.MaxPooling1D(pool_size=2, strides=2)(CG10)

# flat2 = tf.keras.layers.Flatten()(MG5)


### Unificación de redes

In [20]:
# merge input models
# merge = tf.keras.layers.concatenate([flat2])

# interpretation model
hidden1 = tf.keras.layers.Dense(256, activation='relu')(MG5)
hidden2 = tf.keras.layers.Dense(256, activation='relu')(hidden1)
hidden3 = tf.keras.layers.Dense(256, activation='relu')(hidden2)
hidden4 = tf.keras.layers.Dense(256, activation='relu')(hidden3)

In [21]:
output = tf.keras.layers.Dense(n_outputs, activation='tanh')(hidden4)

model = tf.keras.Model(inputs=inputGlobalView, outputs=output)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(4266, 1001, 1)]         0         
_________________________________________________________________
conv1d (Conv1D)              (4266, 997, 16)           96        
_________________________________________________________________
conv1d_1 (Conv1D)            (4266, 993, 16)           1296      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (4266, 496, 16)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (4266, 492, 32)           2592      
_________________________________________________________________
conv1d_3 (Conv1D)            (4266, 488, 32)           5152      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (4266, 244, 32)          

In [22]:
# divisores 4266: 1, 2, 3, 6, 9, 18, 27, 54, 79, 158, 237, 474, 711, 1422, 2133, 4266
bs = 237
Ajuste = model.fit([x_global_extended_train], y_global_extended_train, epochs=50, batch_size=bs)

Epoch 1/50
18/18 [==============================] - 21s 1s/step - loss: 0.6217 - accuracy: 0.7237
Epoch 2/50
18/18 [==============================] - 31s 2s/step - loss: 0.5265 - accuracy: 0.7579
Epoch 3/50
18/18 [==============================] - 29s 2s/step - loss: 0.5034 - accuracy: 0.7553
Epoch 4/50
18/18 [==============================] - 27s 2s/step - loss: 0.4807 - accuracy: 0.7592
Epoch 5/50
18/18 [==============================] - 34s 2s/step - loss: 0.4642 - accuracy: 0.7680
Epoch 6/50
18/18 [==============================] - 27s 1s/step - loss: 0.4560 - accuracy: 0.7743
Epoch 7/50
18/18 [==============================] - 27s 2s/step - loss: 0.4530 - accuracy: 0.7803
Epoch 8/50
18/18 [==============================] - 34s 2s/step - loss: 0.4543 - accuracy: 0.7809
Epoch 9/50
18/18 [==============================] - 37s 2s/step - loss: 0.4541 - accuracy: 0.7785
Epoch 10/50
18/18 [==============================] - 29s 2s/step - loss: 0.4439 - accuracy: 0.7839
Epoch 11/50
18/18 [

In [27]:
y_pred_keras = model.predict(x_global_extended_test).ravel()
display(y_pred_keras)

InvalidArgumentError:  Input to reshape is a tensor with 188416 values, but the requested shape has 25118208
	 [[node functional_1/dense/Tensordot/Reshape (defined at <ipython-input-25-7b0ecb4b8443>:1) ]] [Op:__inference_predict_function_4855]

Function call stack:
predict_function


In [ ]:
# Defino las métricas
fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_local_extended_test, y_pred_keras)
auc_keras = auc(fpr_keras, tpr_keras)
gmeans = np.sqrt(tpr_keras * (1-fpr_keras))

# localiza el índice del mayor g-mean
ix = argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds_keras[ix], gmeans[ix]))


# "#############"


In [ ]:
#Umbral estándar
labels_Standart = (y_pred_keras >= 0.5).astype(np.int)
PhiM_Standart = matthews_corrcoef(y_local_extended_test, labels_Standart)
Matrix_Standart = confusion_matrix(y_local_extended_test, labels_Standart)

#Imprimo por pantalla las métricas
auc_keras, PhiM_Standart, print(Matrix_Standart)


In [ ]:
#Umbral OPT
labels_OPT =(y_pred_keras >= thresholds_keras[ix]).astype(np.int)
PhiCoeff_OPT = matthews_corrcoef(y_local_extended_test, labels_OPT)
Matrix_OPT = confusion_matrix(y_local_extended_test, labels_OPT)
F_Measure_OPT = f1_score(y_local_extended_test, labels_OPT, average='binary')
Accuracy_OPT= accuracy_score(y_local_extended_test, labels_OPT, normalize=True)
Recall_OPT= recall_score(y_local_extended_test, labels_OPT, average=None)
Average_precision_OPT= average_precision_score(y_local_extended_test, labels_OPT)


#Imprimo por pantalla las métricas
print("AUC:", auc_keras)
print("F_Measure:", F_Measure_OPT)
print("PhiCoeff:", PhiCoeff_OPT)
print("Average_precision:", Average_precision_OPT)
print("Accuracy:", Accuracy_OPT)
print("Recall:", Recall_OPT)
print("Matriz de confusión:")
print( Matrix_OPT)


In [ ]:
#Ploteo la matrix de confusión
import seaborn as sn
df_cm = pd.DataFrame(Matrix_OPT, index = [i for i in "01"],
                  columns = [i for i in "01"])

plt.figure(0.5)
heat_map = sn.heatmap(df_cm, xticklabels=True, yticklabels=True, annot=True, annot_kws = {"ha": 'center',"va": 'bottom'},cbar=False)
#heat_map.set_yticklabels(heat_map.get_yticklabels(), rotation=0)
plt.xlabel('Predicted')
plt.ylabel('Actual')

for text in heat_map.texts:
    text.set_size(14)
    text.set_weight('bold')
    if text.get_text() == '5':
        text.set_color('black')


In [ ]:
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='CNN (area = {:.3f})'.format(auc_keras))
plt.scatter(fpr_keras[ix], tpr_keras[ix], marker='o', color='black', label='Best')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')


In [ ]:
y_pred_keras_tot = np.array([])
y_pred_rf_tot = np.array([])
y_test_total = np.array([])

x_local_train, x_local_test, y_local_train, y_local_test = train_test_split(LocalView.iloc[:,2:], LocalView.iloc[:,1], test_size=0.3, random_state=11)
x_global_train, x_global_test, y_global_train, y_global_test = train_test_split(GlobalView.iloc[:,2:], GlobalView.iloc[:,1], test_size=0.3, random_state=11)
x_stellar_train, x_stellar_test, y_stellar_train, y_stellar_test = train_test_split(StellarParam6P.iloc[:,2:], StellarParam6P.iloc[:,1], test_size=0.3, random_state=11)
    
   
    #Definimos las dimensiones
n_outputs = 1
n_local_timesteps, n_local_features  = x_local_train.shape[0], x_local_train.shape[1]
n_global_timesteps, n_global_features  = x_global_train.shape[0], x_global_train.shape[1]
nS_timesteps, nS_features  = x_stellar_train.shape[0], x_stellar_train.shape[1]
      
    #Expandimos las dimensiones
x_local_extended_train = np.expand_dims(x_local_train,axis=-1) # axis=-3 para que se adapte a la forma del tensor
y_local_extended_train = np.array(y_local_train) # Hay que trasponer 
x_global_extended_train = np.expand_dims(x_global_train,axis=-1)
y_global_extended_train = np.array(y_global_train)
x_stellar_extended_train = np.expand_dims(x_stellar_train,axis=-1)
y_stellar_extended_train = np.array(y_stellar_train)
    
    
    # Multiple Inputs

    # first input model
inputLocalView = Input(shape=(n_local_features, 1))
inputLocalView.set_shape([n_local_timesteps, n_local_features, 1])

CL1 = Conv1D(filters=16, kernel_size=5, activation='relu')(inputLocalView)
CL2 = Conv1D(filters=16, kernel_size=5, activation='relu')(CL1)

ML1 = MaxPooling1D(pool_size=7, strides=2)(CL2)

CL3 = Conv1D(filters=32, kernel_size=5, activation='relu')(ML1)
CL4 = Conv1D(filters=32, kernel_size=5, activation='relu')(CL3)

ML2 = MaxPooling1D(pool_size=7, strides=2)(CL4)
flat1 = Flatten()(ML2)

    # second input model

inputGlobalView = Input(shape=(n_global_features, 1))
inputGlobalView.set_shape([n_global_timesteps, n_global_features, 1])

CG1 = Conv1D(filters=16, kernel_size=5, activation='relu')(inputGlobalView)
CG2 = Conv1D(filters=16, kernel_size=5, activation='relu')(CG1)

MG1 = MaxPooling1D(pool_size=3, strides=2)(CG2)

CG3 = Conv1D(filters=32, kernel_size=5, activation='relu')(MG1)
CG4 = Conv1D(filters=32, kernel_size=5, activation='relu')(CG3)

MG2 = MaxPooling1D(pool_size=2, strides=2)(CG4)

CG5 = Conv1D(filters=64, kernel_size=5, activation='relu')(MG2)
CG6 = Conv1D(filters=64, kernel_size=5, activation='relu')(CG5)

MG3 = MaxPooling1D(pool_size=2, strides=2)(CG6)

CG7 = Conv1D(filters=128, kernel_size=3, activation='relu')(MG3)
CG8 = Conv1D(filters=128, kernel_size=3, activation='relu')(CG7)

MG4 = MaxPooling1D(pool_size=2, strides=2)(CG8)

flat2 = Flatten()(MG4)

    # third input model

StellarParam6PInput = Input(shape=(nS_features, 1))
StellarParam6PInput.set_shape([nS_timesteps, nS_features, 1])

    #StellarParam6PInput=(nS_features,1)
flat3 = Flatten()(StellarParam6PInput)
    # merge input models
merge = concatenate([flat1, flat2, flat3])

    # interpretation model
hidden1 = Dense(256, activation='relu')(merge)
hidden2 = Dense(256, activation='relu')(hidden1)
hidden3 = Dense(256, activation='relu')(hidden2)
hidden4 = Dense(256, activation='relu')(hidden3)
 #2output = Dense(n_outputs, activation='tanh')(hidden4)
output = Dense(n_outputs, activation='tanh')(hidden4)
model = Model(inputs=[inputLocalView, inputGlobalView,StellarParam6PInput], outputs=output)
    #loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'ce']
model.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['accuracy'])
    #loss_weights=['main_output': 1., 'aux_output': 0.2]) por si queremos darles pesos diferentes

batch_size=119

#Aumento número de muestras
for i in range(10):
    
    print(i)
    
    # Divido entre entrenamiento y test
    x_local_train, x_local_test, y_local_train, y_local_test = train_test_split(LocalView.iloc[:,2:], LocalView.iloc[:,1], test_size=0.3, random_state=11+i)
    x_global_train, x_global_test, y_global_train, y_global_test = train_test_split(GlobalView.iloc[:,2:], GlobalView.iloc[:,1], test_size=0.3, random_state=11+i)
    x_stellar_train, x_stellar_test, y_stellar_train, y_stellar_test = train_test_split(StellarParam6P.iloc[:,2:], StellarParam6P.iloc[:,1], test_size=0.3, random_state=11+i)
    
  
    # fit forma 2
    FIT = model.fit([x_local_extended_train, x_global_extended_train, x_stellar_extended_train], y_local_extended_train,
          epochs=50, batch_size=batch_size, verbose=0)
    
    x_local_extended_test = np.expand_dims(x_local_test,axis=-1)
    y_local_extended_test = np.array(y_local_test) 
    x_global_extended_test = np.expand_dims(x_global_test,axis=-1)
    y_global_extended_test = np.array(y_global_test)
    x_stellar_extended_test = np.expand_dims(x_stellar_test,axis=-1)
    y_stellar_extended_test = np.array(y_stellar_test)
    
    
    # Creo la RoC
    y_pred_kerasC = model.predict([x_local_extended_test,x_global_extended_test,x_stellar_extended_test]).ravel()
    
    
    #Lleno los vectores totales
    y_pred_keras_tot = np.concatenate((y_pred_keras_tot,y_pred_kerasC))
    
    y_test_total = np.concatenate((y_test_total,y_local_extended_test))


In [ ]:
# Defino las métricas

fpr_kerasTotal, tpr_kerasTotal, thresholds_kerasTotal = roc_curve(y_test_total, y_pred_keras_tot)
auc_kerasTotal = auc(fpr_kerasTotal, tpr_kerasTotal)
gmeans = np.sqrt(tpr_kerasTotal * (1-fpr_kerasTotal))
# localiza el índice del mayor g-mean
ix = argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds_kerasTotal[ix], gmeans[ix]))


In [ ]:
#Umbral estándar
labels_Standart = (y_pred_keras_tot >= 0.5).astype(np.int)
PhiCoeff_Standart = matthews_corrcoef(y_test_total, labels_Standart)
Matrix_Standart = confusion_matrix(y_test_total, labels_Standart)
F_Measure_Standart = f1_score(y_test_total, labels_Standart, average='binary')

#Imprimo por pantalla las métricas
auc_kerasTotal, PhiM_Standart,F_Measure_Standart, print(Matrix_Standart)


In [ ]:
#Umbral OPT
labels_OPT =(y_pred_keras_tot >= thresholds_kerasTotal[ix]).astype(np.int)
PhiCoeff_OPT = matthews_corrcoef(y_test_total, labels_OPT)
Matrix_OPT = confusion_matrix(y_test_total, labels_OPT)
F_Measure_OPT = f1_score(y_test_total, labels_OPT, average='binary')
Accuracy_OPT= accuracy_score(y_test_total, labels_OPT, normalize=True)
Recall_OPT= recall_score(y_test_total, labels_OPT, average=None)
Average_precision_OPT= average_precision_score(y_test_total, labels_OPT)


#Imprimo por pantalla las métricas
print("AUC:", auc_kerasTotal)
print("F_Measure:", F_Measure_OPT)
print("PhiCoeff:", PhiCoeff_OPT)
print("Average_precision:", Average_precision_OPT)
print("Accuracy:", Accuracy_OPT)
print("Recall:", Recall_OPT)
print("Matriz de confusión:")
print( Matrix_OPT)


In [ ]:
#Ploteo la matrix de confusión

df_cm_OPT = pd.DataFrame(Matrix_OPT, index = [i for i in "01"],
                  columns = [i for i in "01"])
plt.figure(0.5)
heat_map = sn.heatmap(df_cm_OPT, xticklabels=True, yticklabels=True, annot=True, fmt='g', annot_kws = {"ha": 'center',"va": 'bottom'},cbar=False)
#heat_map.set_yticklabels(heat_map.get_yticklabels(), rotation=0)
plt.xlabel('Predicted')
plt.ylabel('Actual')

for text in heat_map.texts:
    text.set_size(14)
    text.set_weight('bold')
    if text.get_text() == '33':
        text.set_color('black')


In [ ]:
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_kerasTotal, tpr_kerasTotal,  color='green', label='Tanh/Adadelta = {:.3f})'.format(auc_kerasTotal))
plt.scatter(fpr_kerasTotal[ix], tpr_kerasTotal[ix], marker='o', color='black', label='Best')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()